In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error
import numpy as np


Reading only the following columns => `wdir` | `Tper` | `sper` | `Tdir` | `sdir` | `Length` | `Width`

In [163]:
fields = [
    'whgt',
    'wper',
    'wdir',
    'Tper',
    'sper',
    'Tdir',
    'sdir',
    # 'Length',
    # 'Width',
    "SOG",
    "VesselType"
]

In [164]:
# reading selective columns for our model
df = pd.read_csv("./Draft Cleaned Merged.csv", skipinitialspace=True, usecols=fields)
df.head()

,Tdir,Tper,sdir,sper,wdir,wper,whgt,SOG,VesselType
0,198.0,6.024096,213.0,5.9,296.0,2.49,0.28,9.7,80.0
1,198.0,6.024096,213.0,5.9,296.0,2.49,0.28,9.7,80.0
2,198.0,6.024096,213.0,5.9,296.0,2.49,0.28,9.8,80.0
3,198.0,6.024096,213.0,5.9,296.0,2.49,0.28,9.8,80.0
4,198.0,6.024096,213.0,5.9,296.0,2.49,0.28,9.9,80.0


In [165]:
df = df[df["SOG"] > 0.0]

In [166]:
df.shape

(4402, 9)

In [167]:
Y = df[["SOG"]]
X = df.drop("SOG", axis=1)

In [168]:
# cross validation technique being applied => test-train split
x_train, x_test, y_train, y_test= train_test_split(X, Y, test_size= 0.25, random_state=0)

In [169]:
x_train.shape

(3301, 8)

In [170]:
# # importing r2_score module
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_squared_error
# # predicting the accuracy score
# score=r2_score(y_test, y_pred)
# print("r2 socre is ",score)
# print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
# print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))

In [171]:
# Fitting Polynomial Regression to the dataset
from sklearn.preprocessing import PolynomialFeatures
poly_model = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly_model.fit_transform(X)

In [172]:
# cross validation technique being applied => test-train split
x_train, x_test, y_train, y_test= train_test_split(poly_features, Y, test_size= 0.25, random_state=0)

In [173]:
# we use linear regression as a base
poly_reg_model = LinearRegression()
poly_reg_model.fit(x_train, y_train)
poly_reg_model.score(x_test, y_test)

0.6958324663619755

In [174]:
X.head(100)

,Tdir,Tper,sdir,sper,wdir,wper,whgt,VesselType
0,198.0,6.024096,213.0,5.90,296.0,2.49,0.28,80.0
1,198.0,6.024096,213.0,5.90,296.0,2.49,0.28,80.0
2,198.0,6.024096,213.0,5.90,296.0,2.49,0.28,80.0
3,198.0,6.024096,213.0,5.90,296.0,2.49,0.28,80.0
4,198.0,6.024096,213.0,5.90,296.0,2.49,0.28,80.0
...,...,...,...,...,...,...,...,...
95,180.0,6.896551,212.0,6.27,307.0,2.70,0.37,80.0
96,180.0,6.896551,212.0,6.27,307.0,2.70,0.37,80.0
97,180.0,6.896551,212.0,6.27,307.0,2.70,0.37,80.0
98,180.0,6.896551,212.0,6.27,307.0,2.70,0.37,80.0


In [175]:
Y.head(100)

,SOG
0,9.7
1,9.7
2,9.8
3,9.8
4,9.9
...,...
95,9.9
96,9.9
97,9.9
98,9.9


In [180]:
poly_reg_model.predict(poly_model.fit_transform([[
    180.0,	6.896551,	212.0,	6.27,	307.0,	2.70,	0.37, 80,
]]))

array([[9.884375]])

In [ ]:
temp = np.array([
    198,
    6.02,
    213,
    5.9,
    296
]).reshape(1, -1)
predict_ = poly_model.fit_transform(temp)
predict_[0].shape
# predict_ = np.delete(predict_,(1),axis=1)

poly_reg_model.predict(predict_)

ValueError: X has 55 features, but LinearRegression is expecting 119 features as input.

In [ ]:
poly_features.shape
poly_features

array([[1.98000000e+02, 6.02409600e+00, 2.13000000e+02, ...,
        1.03037600e+04, 5.16934400e+05, 2.59343360e+07],
       [1.98000000e+02, 6.02409600e+00, 2.13000000e+02, ...,
        1.03037600e+04, 5.16934400e+05, 2.59343360e+07],
       [1.98000000e+02, 6.02409600e+00, 2.13000000e+02, ...,
        1.03037600e+04, 5.16934400e+05, 2.59343360e+07],
       ...,
       [7.50000000e+01, 8.47457600e+00, 7.10000000e+01, ...,
        2.45714373e+04, 9.52536510e+05, 3.69260370e+07],
       [7.50000000e+01, 8.47457600e+00, 7.10000000e+01, ...,
        2.45714373e+04, 9.52536510e+05, 3.69260370e+07],
       [7.50000000e+01, 8.47457600e+00, 7.10000000e+01, ...,
        2.45714373e+04, 9.52536510e+05, 3.69260370e+07]])

In [ ]:
y_predicted = poly_reg_model.predict(poly_features)
y_predicted

array([[ 9.8906224 ],
       [ 9.8906224 ],
       [ 9.8906224 ],
       ...,
       [12.52739639],
       [12.52739639],
       [12.52739639]])

In [ ]:
temp = np.array([
   211.3,
    6.04,
    213,
    5.9,
    297
]).reshape(1, -1)
predict_ = poly_model.fit_transform(temp)
predict_[0].shape
# predict_ = np.delete(predict_,(1),axis=1)

regression_model.predict(predict_)

ValueError: X has 55 features, but LinearRegression is expecting 792 features as input.

In [ ]:
# importing r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_predicted)
print('r2 score is ',score)
print('mean_sqrd_error is==',mean_squared_error(y_test,y_predicted))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_predicted)))

ValueError: Found input variables with inconsistent numbers of samples: [1101, 3301]

In [ ]:
# # check our accuracy for each degree, the lower the error the better!
# number_degrees = [1,2,3,4,5,6,7]
# plt_mean_squared_error = []
# for degree in number_degrees:

#    poly_model = PolynomialFeatures(degree=degree)
  
#    poly_x_values = poly_model.fit_transform(X.values)
#    # poly_model.fit(poly_x_values, Y.values)
  
#    regression_model = LinearRegression()
#    regression_model.fit(poly_x_values, Y.values)
#    y_pred = regression_model.predict(poly_x_values)
  
#    plt_mean_squared_error.append(mean_squared_error(Y.values, y_pred, squared=False))
  
# plt.scatter(number_degrees,plt_mean_squared_error, color="green")
# plt.plot(number_degrees,plt_mean_squared_error, color="red") 